In [ ]:
!pip install openai

In [3]:
import os
os.environ['OPENAI_API_KEY'] = "..."

In [4]:
import sqlite3

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルを作成する
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT
)
''')

# productsテーブルを作成する
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price INTEGER
)
''')

# order_historyテーブルを作成する
cursor.execute('''
CREATE TABLE order_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    purchase_date DATE,
    quantity INTEGER,
    remarks TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

# support_historyテーブルを作成する
cursor.execute('''
CREATE TABLE support_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    subject TEXT,
    message_content TEXT,
    message_type TEXT,
    status TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [5]:
import sqlite3
from datetime import datetime

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルに日本語のテストデータを挿入する
users_data = [
    (1, '太郎', '山田', 'taro@example.com', '090-1234-5678'),
    (2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')
]

for user in users_data:
    cursor.execute('''
    INSERT INTO users (user_id, first_name, last_name, email, phone)
    VALUES (?, ?, ?, ?, ?)
    ''', user)

# productsテーブルに日本語のテストデータを挿入する
products_data = [
    (1, '商品A', 1000),
    (2, '商品B', 2000),
    (3, '商品C', 3000)
]

for product in products_data:
    cursor.execute('''
    INSERT INTO products (product_id, product_name, price)
    VALUES (?, ?, ?)
    ''', product)

# order_historyテーブルに日本語のテストデータを挿入する
order_history_data = [
    (1, 1, 1, '2023-04-01', 2, '迅速な発送', datetime.now()),
    (2, 2, 3, '2023-04-05', 1, 'ギフトラッピング', datetime.now())
]

for order in order_history_data:
    cursor.execute('''
    INSERT INTO order_history (history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', order)

# support_historyテーブルに日本語のテストデータを挿入する
support_history_data = [
    (1, 1, '請求に関する問題', None, None, 'open', datetime.now()),
    (2, 1, None, '請求に問題があります。', 'user', None, datetime.now()),
    (3, 1, None, 'お問い合わせいただきありがとうございます。問題を調査しています。', 'support', None, datetime.now())
]

for support in support_history_data:
    cursor.execute('''
    INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', support)

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [7]:
import sqlite3

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# 各テーブルからデータを取得して表示する
tables = ['users', 'products', 'order_history', 'support_history']

for table in tables:
    print(f"{table} テーブルのデータ:")
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    print()  # 空行を挿入して見やすくする

# データベース接続を閉じる
conn.close()

users テーブルのデータ:
(1, '太郎', '山田', 'taro@example.com', '090-1234-5678')
(2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')

products テーブルのデータ:
(1, '商品A', 1000)
(2, '商品B', 2000)
(3, '商品C', 3000)

order_history テーブルのデータ:
(1, 1, 1, '2023-04-01', 2, '迅速な発送', '2023-06-07 13:18:31.806857')
(2, 2, 3, '2023-04-05', 1, 'ギフトラッピング', '2023-06-07 13:18:31.806861')

support_history テーブルのデータ:
(1, 1, '請求に関する問題', None, None, 'open', '2023-06-07 13:18:31.807199')
(2, 1, None, '請求に問題があります。', 'user', None, '2023-06-07 13:18:31.807202')
(3, 1, None, 'お問い合わせいただきありがとうございます。問題を調査しています。', 'support', None, '2023-06-07 13:18:31.807203')



In [10]:
import openai
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError
from tabulate import tabulate
from json.decoder import JSONDecodeError
import json
import re


engine = create_engine('sqlite:///user_support.db', echo=False)


# Predefined system message
SYSTEM_MESSAGE = """
次のように定義されたテーブルがあります。
定義データ:
{meta_text}

これらの定義データにもとづいてユーザからの要求文にもっとも適したSQL文を生成してください。
次のJSON形式でのみ出力してください。説明は100字以内に収めてください。

{{
    "sql": (SQL文),
    "description": (説明)
}}

## 会話の例
USER->ユーザ一覧を出力してください。
AI->{{"sql" : "SELECT user_id, first_name, last_name, email, phone FROM users","description" : "ユーザ一覧を取得するために、usersテーブルから必要な情報を取得するSELECT文を生成しました。"}}

"""

def create_sql_response(text, meta_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature = 0.3,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_MESSAGE.format(meta_text=meta_text)
            },
            {
                "role": "user",
                "content": f"要求文: {text}...結果はJSON形式で出力してください"
            }
        ],
    )
    try:
      print(response.choices[0].message['content'])
      json_str = re.search(r'\{.*\}', response.choices[0].message['content'],re.DOTALL).group(0)
      return json.loads(json_str)
    except JSONDecodeError:
      return {
          "sql": None,
          "description": None
      }

def get_meta_data():
    metadata = MetaData()
    metadata.reflect(bind=engine)

    meta_text = ''
    for table in metadata.tables.values():
        meta_text += 'テーブル名:'+ table.name + '\n'
        for column in table.columns:
            meta_text += f'列名:{column.name}, 型: {column.type}\n'
        meta_text += '\n'
    return meta_text

def exec_sql(sql):
    Session = sessionmaker(bind=engine)

    try:
        with Session() as session:
            t = text (sql)
            result = session.execute(t)

            header = [k for k in result.keys()]

            rows = result.fetchall()
            tabled = tabulate(rows,header, tablefmt="github")
            print(tabled)

    except SQLAlchemyError as e:
        print(f'Exception Excute SQL: {e}\n')

def repl():
    meta_text = get_meta_data() 
    if not meta_text:
        print("メタデータが入力されていません")
        return
    print('メタデータが読み込まれました\n')
    while True:
        try:
          user_input = input(">")
          if user_input:
              response = create_sql_response(user_input, meta_text)
              sql = response['sql']
              description = response['description']
              
              print('SQL文:\n', sql)
              print('説明:\n', description)
              print()
              if sql:
                exec_sql(sql)
                
        except (KeyboardInterrupt, EOFError):
            print()
            break

if __name__ == "__main__":
    repl()

メタデータが読み込まれました

>ユーザ一覧
{"sql" : "SELECT user_id, first_name, last_name, email, phone FROM users","description" : "ユーザ一覧を取得するために、usersテーブルから必要な情報を取得するSELECT文を生成しました。"}
SQL文:
 SELECT user_id, first_name, last_name, email, phone FROM users
説明:
 ユーザ一覧を取得するために、usersテーブルから必要な情報を取得するSELECT文を生成しました。

|   user_id | first_name   | last_name   | email              | phone         |
|-----------|--------------|-------------|--------------------|---------------|
|         1 | 太郎         | 山田        | taro@example.com   | 090-1234-5678 |
|         2 | 花子         | 佐藤        | hanako@example.com | 080-9876-5432 |
>quit
AI->{"sql" : null, "description" : "ユーザからの要求が終了したため、SQL文は生成されませんでした。"}
SQL文:
 None
説明:
 ユーザからの要求が終了したため、SQL文は生成されませんでした。

>

